# GR5291 YouTube Trending Analysis 
# Group 5
# Feature Engineering And Random Forest Model Baseline

## 1. Import Packages and Read Data

In [83]:
### Install Packages

import os
# ! pip install plotly
# ! pip install -U gensim
# ! python -m pip install --upgrade pip
# ! pip install wordcloud
! pip3 install --upgrade pandas

    100% |████████████████████████████████| 8.9MB 2.8MB/s 
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [0]:
### Import Packages

import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from sklearn import preprocessing
import time
import datetime
# import lightgbm as lgb
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from collections import Counter
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
# nltk.download()
np.random.seed(2018)
pd.options.display.max_columns = 999

In [239]:
### ignore this block when using local Jupyter Notebook (Anaconda), used to upload files in Google Colab

from google.colab import files
upload1 = files.upload()
# upload2 = files.upload()

Saving df_no_display3.csv to df_no_display3.csv


In [0]:
### read csv and json data

### USvideos raw csv data
USvideos = pd.read_csv("USvideos.csv")

### US category id raw json data
UScat = pd.read_json("US_category_id.json")

### the description2vector data from Xinrou
ssv = pd.read_csv("simplified_sentence_vector2.csv")
# ssv_new = pd.read_csv('simplified_sentence_vector3.txt', sep=" ", header=None)

### if the description is NaN in original data, then use the tags and titles to convert vector instead
description_NaN = pd.read_csv("description_NaN.csv")

### some vectors are not displayed due to the long length of the description, and we store these vectors individually
df_no_display = pd.read_csv("df_no_display3.csv")

In [331]:
USvideos.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...


## 2. Preprocessing
### 2.1 processing date
#### 2.1.1 convert to standard time

In [0]:
USvideos.category_id = USvideos.category_id.astype('category')
USvideos['trending_date'] = pd.to_datetime(USvideos['trending_date'], format='%y.%d.%m').dt.date
# Transforming publish_time to datetime
publish_time = pd.to_datetime(USvideos.publish_time, format='%Y-%m-%dT%H:%M:%S.%fZ')

# Create Variable publish_date
USvideos['publish_date'] = publish_time.dt.date

# Drop publish_time
USvideos.drop('publish_time',axis=1,inplace=True)

#### 2.1.2 only keep data with trending dates within 31 days after publish date

In [333]:
cond1 = USvideos['trending_date'] < USvideos['publish_date']+timedelta(days=31)
cond2 = USvideos['trending_date'] >= USvideos['publish_date']
cond = cond1 & cond2
sum(cond)
USvideos2 = USvideos[cond]
USvideos2.shape

(40584, 16)

#### 2.1.3 Add Column: compute the number of days on trending

In [334]:
USvideos2['date_delta'] = (USvideos2['trending_date'] - USvideos2['publish_date']).map(lambda x: x.days)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### 2.2 Create new Data Frame

for each video, we only keep the data of its first trending date 

In [335]:
first_trend_df = USvideos2.sort_values(['video_id', 'date_delta'], ascending = True).groupby('video_id').first()
first_trend_df.reset_index(level=0, inplace=True)
first_trend_df.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2


In [336]:
first_trend_df.shape ### it is as expected, as the number of rows == number of unique video ids 

(6250, 17)

### 2.3 Merge the vector data to our current dataframe

1. In another Python Notebook, we convert the descriptions to 25-dimensional vectors, named d1 to d25, using n-grams and stored the vector data as tfidf_sentence_vector.csv. 
2. However, the description column for some videos are empty, and in this case, we choose to use the data converted from tags and titles instead. The vector data of these videoes are called description_NaN.csv.

In [0]:
### simplified_sentence_vector data from XinRou, 25 dimensions
ssv2 = ssv.drop(['tfidf_sentence_vector', 'Unnamed: 0'], axis = 1)
ssv2 = ssv2.dropna()

In [338]:
mergedf = pd.merge(first_trend_df, ssv2, left_on = ["video_id" , "description"], right_on = ["video_id" , "description"], how = 'left')
mergedf.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848


In [339]:
mergedf_null = mergedf.loc[mergedf['d0'].isnull(), ]
mergedf_notnull = mergedf.loc[mergedf['d0'].notnull(), ]
mergedf_null.shape

(100, 42)

In [0]:
### description_NaN data is for those rows without descriptions, and it contains the vectors converted from tags & title (bag of words)
description_NaN2 = description_NaN.drop(['title_tags', 'title_tags_vector', 'Unnamed: 0'], axis = 1)
description_NaN2.head(1)


drop_col = []
for i in range(25):
  drop_col.append('d' + str(i))

mergedf_null = mergedf_null.drop(drop_col, axis = 1)

mergedf_null = pd.merge(mergedf_null, description_NaN2, left_on = ["video_id"], right_on = ["video_id"], how = 'left')

In [341]:
mergedf_null2 = mergedf_null.loc[mergedf_null['d0'].isnull(), ]
mergedf_notnull2 = mergedf_null.loc[mergedf_null['d0'].notnull(), ]
mergedf_null2.shape

(16, 42)

In [342]:
df_no_display.columns

Index(['Unnamed: 0', 'video_id', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6',
       'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16',
       'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24'],
      dtype='object')

In [0]:
df_no_display2 = df_no_display.drop(['Unnamed: 0'], axis = 1)
df_no_display2 = df_no_display2.drop_duplicates(subset = ['video_id'])
df_no_display2.head(1)


drop_col = []
for i in range(25):
  drop_col.append('d' + str(i))

mergedf_null2 = mergedf_null2.drop(drop_col, axis = 1)

mergedf_null2 = pd.merge(mergedf_null2, df_no_display2, left_on = ["video_id"], right_on = ["video_id"], how = 'left')

In [344]:
mergedf = pd.concat([mergedf_notnull, mergedf_notnull2, mergedf_null2], axis = 0)
mergedf = mergedf.reset_index(drop=True)
mergedf.shape

(6250, 42)

In [345]:
mergedf.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848


#### 2.4 Add Column: Count the appearance time of each video within 31 days

In [346]:
first_trend_df2 = mergedf
appearance_time = USvideos2.groupby('video_id')['trending_date'].nunique()
first_trend_df2['app_time'] = first_trend_df2['video_id'].map(lambda x: appearance_time[x])
first_trend_df2.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,app_time
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,6


### 2.5 Add Column: Count the number of tags

In [347]:
first_trend_df2['tags2'] = [tag.split("|") for tag in first_trend_df['tags']]
first_trend_df2['tagsLength'] = first_trend_df2['tags2'].apply(len)
first_trend_df2.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,app_time,tags2,tagsLength
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,6,"[ellen, ""ellen degeneres"", ""the ellen show"", ""...",33


### 2.6 Process Category ID
#### 2.6.1 Category ID to Category Name

In [348]:
categories = {int(category['id']): category['snippet']['title'] for category in UScat['items']}
catDict = {}
counter = 0
for k,v in categories.items():
    catDict[k] = v
    counter += 1
catDict

{1: 'Film & Animation',
 2: 'Autos & Vehicles',
 10: 'Music',
 15: 'Pets & Animals',
 17: 'Sports',
 18: 'Short Movies',
 19: 'Travel & Events',
 20: 'Gaming',
 21: 'Videoblogging',
 22: 'People & Blogs',
 23: 'Comedy',
 24: 'Entertainment',
 25: 'News & Politics',
 26: 'Howto & Style',
 27: 'Education',
 28: 'Science & Technology',
 29: 'Nonprofits & Activism',
 30: 'Movies',
 31: 'Anime/Animation',
 32: 'Action/Adventure',
 33: 'Classics',
 34: 'Comedy',
 35: 'Documentary',
 36: 'Drama',
 37: 'Family',
 38: 'Foreign',
 39: 'Horror',
 40: 'Sci-Fi/Fantasy',
 41: 'Thriller',
 42: 'Shorts',
 43: 'Shows',
 44: 'Trailers'}

In [349]:
first_trend_df2['category'] = first_trend_df2['category_id'].map(lambda x: catDict[x])
first_trend_df2.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,app_time,tags2,tagsLength,category
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,6,"[ellen, ""ellen degeneres"", ""the ellen show"", ""...",33,Entertainment


#### 2.6.2 Convert Category to Dummy Variables

In [0]:
dummy_category = pd.get_dummies(first_trend_df2['category'])

In [351]:
first_trend_df3 = pd.concat([first_trend_df2, dummy_category], axis=1)
first_trend_df3.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,app_time,tags2,tagsLength,category,Autos & Vehicles,Comedy,Education,Entertainment,Film & Animation,Gaming,Howto & Style,Music,News & Politics,Nonprofits & Activism,People & Blogs,Pets & Animals,Science & Technology,Shows,Sports,Travel & Events
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,6,"[ellen, ""ellen degeneres"", ""the ellen show"", ""...",33,Entertainment,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


### 2.7 Hot Channels

In [352]:
countChannel = {}
for i in range(first_trend_df3.shape[0]):
    channel = first_trend_df3['channel_title'][i]
    countChannel[channel.lower()] = countChannel.get(channel.lower(), 0) + 1
    
top10Channel = Counter(countChannel).most_common()[:10]
top10Channel = [x[0] for x in top10Channel]
first_trend_df3['top10Channel'] = first_trend_df3['channel_title'].map(lambda x: x in top10Channel)

top100Channel = Counter(countChannel).most_common()[:100]
top100Channel = [x[0] for x in top100Channel]
first_trend_df3['top100Channel'] = first_trend_df3['channel_title'].map(lambda x: x in top100Channel)

top500Channel = Counter(countChannel).most_common()[:500]
top500Channel = [x[0] for x in top500Channel]
first_trend_df3['top500Channel'] = first_trend_df3['channel_title'].map(lambda x: x in top500Channel)

first_trend_df3.head(1)

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,app_time,tags2,tagsLength,category,Autos & Vehicles,Comedy,Education,Entertainment,Film & Animation,Gaming,Howto & Style,Music,News & Politics,Nonprofits & Activism,People & Blogs,Pets & Animals,Science & Technology,Shows,Sports,Travel & Events,top10Channel,top100Channel,top500Channel
0,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2018-06-04,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,6,"[ellen, ""ellen degeneres"", ""the ellen show"", ""...",33,Entertainment,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False


### 2.8 Sentimental Score: In **Progress**

#### 2.9 Output CSV

In [0]:
first_trend_df3.to_csv('first_trend_df.csv', sep=',', encoding='utf-8')

In [368]:
! ls

description_NaN.csv  simplified_sentence_vector2.csv
df_no_display3.csv   simplified_sentence_vector3.txt
first_trend_df.csv   US_category_id.json
model_data.csv	     USvideos.csv
sample_data


In [0]:
files.download('first_trend_df.csv')

### 3. Model Baseline (Random Forest)

#### 3.1 Create Model Data

In [354]:
first_trend_df3.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link',
       'comments_disabled', 'ratings_disabled', 'video_error_or_removed',
       'description', 'publish_date', 'date_delta', 'd0', 'd1', 'd2', 'd3',
       'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24',
       'app_time', 'tags2', 'tagsLength', 'category', 'Autos & Vehicles',
       'Comedy', 'Education', 'Entertainment', 'Film & Animation', 'Gaming',
       'Howto & Style', 'Music', 'News & Politics', 'Nonprofits & Activism',
       'People & Blogs', 'Pets & Animals', 'Science & Technology', 'Shows',
       'Sports', 'Travel & Events', 'top10Channel', 'top100Channel',
       'top500Channel'],
      dtype='object')

### 3.2 Classification Model: after first time on trending board, will the video be trending again within 31 days after published?

#### 3.2.1 Split Train and Test Data

In [399]:
X = first_trend_df3
X = X.drop(['video_id', 'trending_date','title','channel_title','category_id','tags','thumbnail_link','description','publish_date','app_time','tags2','category'], axis = 1)
y = first_trend_df3['app_time'] <= 1 ### 1 for only trending once within 31 days, 0 otherwise
X.head(1)

,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,date_delta,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,tagsLength,Autos & Vehicles,Comedy,Education,Entertainment,Film & Animation,Gaming,Howto & Style,Music,News & Politics,Nonprofits & Activism,People & Blogs,Pets & Animals,Science & Technology,Shows,Sports,Travel & Events,top10Channel,top100Channel,top500Channel
0,475965,6531,172,271,False,False,False,2,-0.161175,0.124738,0.175031,-0.003445,-0.017617,-0.032546,0.114659,-0.125688,-0.062131,-0.110315,0.002753,0.102883,-0.802276,0.012264,0.013673,-0.028617,0.127851,-0.111181,-0.010753,-0.055571,-0.004595,0.067133,0.018896,-0.065106,-0.05848,33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False


Note that the classification is imbalanced:

In [400]:
n_pos = sum(y == 1)
n_neg = sum(y == 0)
print('Number of Positive Case:', n_pos, 'Number of Negative Case:', n_neg)

Number of Positive Case: 692 Number of Negative Case: 5558


In [0]:
np.random.seed(2018)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [402]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:',  y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (5000, 53)
Training Labels Shape: (5000,)
Testing Features Shape: (1250, 53)
Testing Labels Shape: (1250,)


### 3.2.2 Train a Baseline Model (No Cross-Validation, No Tuning Parameter)

In [403]:
clf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

#### 3.2.3 Baseline Results

In [404]:
predictions = clf.predict(X_test)
errors = sum(predictions != y_test)/len(y_test)
print(errors)

0.1096


#### 3.2.4 Feature Importance

In [405]:
clf_feature_importances = pd.Series(clf.feature_importances_, index = X_train.columns)
print(clf_feature_importances.sort_values(ascending = False))

likes                     0.038394
views                     0.038251
date_delta                0.036708
d16                       0.035964
comment_count             0.035763
dislikes                  0.035116
d14                       0.033296
d12                       0.032399
d6                        0.031710
d23                       0.031551
tagsLength                0.031173
d2                        0.030843
d9                        0.030237
d19                       0.030090
d15                       0.030073
d4                        0.029921
d13                       0.029800
d11                       0.029642
d0                        0.029606
d24                       0.029518
d3                        0.029381
d22                       0.029267
d20                       0.029145
d10                       0.029141
d1                        0.028800
d8                        0.028715
d7                        0.028425
d17                       0.028382
d18                 

#### 3.2.5 Confusion Matrix

In [406]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[1106,    5],
       [ 132,    7]])

In [407]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(tn, fp, fn, tp)

1106 5 132 7


#### 3.2.6 Metrics Evaluation

In [408]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
fp_rate = fp / (fp + tn)
f1_score = 2 * (precision * recall / (precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("recall:", recall, "\n precision:", precision, "\n fp_rate:",fp_rate, "\n f1_score:",f1_score, "\n accuracy:",accuracy)

recall: 0.050359712230215826 
 precision: 0.5833333333333334 
 fp_rate: 0.004500450045004501 
 f1_score: 0.09271523178807947 
 accuracy: 0.8904


### 3.3 Regression Model: after first time on trending board, how many times will the video be trending again within 31 days after published?

#### 3.3.1 Split Train and Test Data

In [0]:
X = first_trend_df3
X = X.drop(['video_id', 'trending_date','title','channel_title','category_id','tags','thumbnail_link','description','publish_date','app_time','tags2','category'], axis = 1)
y = first_trend_df3['app_time'] - 1
X.head(1)

In [0]:
np.random.seed(2018)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [373]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:',  y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (5000, 53)
Training Labels Shape: (5000,)
Testing Features Shape: (1250, 53)
Testing Labels Shape: (1250,)


### 3.3.2 Train a Baseline Model (No Cross-Validation, No Tuning Parameter)

In [359]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

#### 3.3.3 Baseline Results

In [360]:
### Use the forest's predict method on the test data
predictions = rf.predict(X_test)

### Calculate the absolute errors
errors = abs(predictions - y_test)

### Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 3.26 degrees.


#### 3.3.4 Feature Importance

In [366]:
rf_feature_importances = pd.Series(rf.feature_importances_, index = X_train.columns)
print(rf_feature_importances.sort_values(ascending = False))

likes                     0.150479
views                     0.055075
dislikes                  0.038349
date_delta                0.037988
d16                       0.036243
comment_count             0.036137
d17                       0.033329
d22                       0.032847
d3                        0.030674
d24                       0.028826
d10                       0.028790
d8                        0.028661
tagsLength                0.028595
d6                        0.027849
d14                       0.027511
d2                        0.025935
d19                       0.024463
d13                       0.024052
d12                       0.022857
d9                        0.021975
d1                        0.021951
d23                       0.021772
d20                       0.021743
d11                       0.021381
d15                       0.021091
d7                        0.020286
d18                       0.019213
d4                        0.018976
d0                  

## 4. Future Work

### 4.1 More feature engineering like the complete sentimental scores (cuurent sentimental score data is incomplete)

### 4.2 Cross-Validation and Tuning Parameters for both classification problem and regression problem

### 4.3 More Models: lightGBM, XGBoost, CatBoost, etc.

### 4.4 Note that the classification problem is facing imbalanced classes problems, consider methods like oversampling, undersampling, classifier cascades, cost-sensitive learning, etc.